## 5.12-RNN-LSTM-例子2-回归

【目的】： 使用到自己创建的 sin 曲线预测一条 cos 曲线. 

【知识点框架】：
- 第一步：定义参数
- 第二步：数据生成
- 第三步：定义LSTM-RNN的主体结构
    1. 定义X和Y
    2. 定义输入层（第一次隐藏层）
    3. 定义cell层
    4. 定义输入层（第二次隐藏层）
    5. 计算损失值
    6. 定义训练器，对损失值进行优化
- 第四步：训练LSTM-RNN

In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow as tf1
import numpy as np
import matplotlib.pyplot as plt

### 第一步：定义参数

In [4]:
BATCH_START = 0
#backpropagation through time 的 time_steps（随时间反向传播间隔）
TIME_STEPS = 20 
BATCH_SIZE = 50
INPUT_SIZE = 1 # SIN数据输入的size
OUTPUT_SIZE = 1 #cos数据输入的size 
CELL_SIZE = 10 #RNN 的hidden unit size
LR = 0.06 #learning rate

### 第二步：数据生成

In [15]:
def get_batch():
    global BATCH_START, TIME_STEPS
    xs = np.arange(BATCH_START, BATCH_START + TIME_STEPS *
                   BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS))/(10 * np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
#     np.newaxis的功能:插入新维度 将2D转3D
    return [seq[:,:,np.newaxis], res[:,:,np.newaxis], xs]

### 第三步：定义LSTM-RNN的主体结构

In [21]:
class LSTMRNN(object):
#     n_step: BP反向传播的步长
    def __init__(self, n_steps, input_size, output_size, cell_size, batch_size):
        self.n_steps = n_steps
        self.input_size = input_size
        self.output_size = output_size
        self.cell_size = cell_size
        self.batch_size = batch_size
        
        with tf.name_scope('inputs'):
            self.xs = tf.placeholder(tf.float32, [None, n_steps, input_size], name='xs')
            self.ys = tf.placeholder(tf.float32, [None, n_steps, output_size], name='ys')
        with tf.variable_scope('in_hidden'):
            self.add_input_layer()
        with tf.name_scope('LSTM_cell'):
            self.add_cell()
        with tf.variable_scope('out_hidden'):
            self.add_output_layer()
        with tf.name_scope('cost'):
            self.compute_cost()
        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(LR).minimize(self.cost)
            
    # 定义输入层（第一次隐藏层）
    def add_input_layer(self):
        # (batch*n_step, in_size)
        l_in_x = tf.reshape(self.xs, [-1,self.input_size], name = '2_2D')
        # Ws(in_size, cell_size)
        Ws_in = self._weight_variable([self.input_size, self.cell_size])
        # bs(cell_size,)
        bs_in = self._bias_variable([self.cell_size,])
        
        # l_in_y = (batch * n_steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            l_in_y = tf.matmul(l_in_x, Ws_in) + bs_in
        self.l_in_y = tf.reshape(l_in_y, [-1, self.n_steps, self.cell_size], name='2_3D')

    #定义cell层
    def add_cell(self):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(self.cell_size, 
                                                 forget_bias=1.0, 
                                                 state_is_tuple=True)
        with tf.name_scope('initial_state'):
            self.cell_init_state = lstm_cell.zero_state(self.batch_size, 
                                                        dtype=tf.float32)
        self.cell_outputs, self.cell_final_state = tf.nn.dynamic_rnn(
            lstm_cell, self.l_in_y, initial_state= self.cell_init_state,
            time_major=False)        
        
    #定义输出层(第二次隐藏层)
    def add_output_layer(self):
        #shape = (batch * steps, cell_size)
        l_out_x = tf.reshape(self.cell_outputs, [-1, self.cell_size], name='2_2D')
        Ws_out = self._weight_variable([self.cell_size, self.output_size])
        bs_out = self._bias_variable([self.output_size, ])
        #shape =(batch * steps, cell_size)
        with tf.name_scope('Wx_plus_b'):
            self.pred = tf.matmul(l_out_x, Ws_out) + bs_out
        
    def compute_cost(self):
        losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
                    [tf.reshape(self.pred, [-1], name = 'reshape_pred')],
                    [tf.reshape(self.ys, [-1], name = 'reshape_target')],
                    [tf.ones([self.batch_size * self.n_steps], dtype=tf.float32)],
                    average_across_timesteps = True,
                    softmax_loss_function = self.ms_error,
                    name='losses')
        with tf.name_scope('average_cost'):
            self.cost = tf.div(
                tf.reduce_sum(losses, name='losses_sum'),
                self.batch_size,
                name='average_cost')
            tf.summary.scalar('cost', self.cost)
        
    @staticmethod
    def ms_error(labels, logits):
        return tf.square(tf.subtract(labels, logits))
    
    def _weight_variable(self, shape, name='weights'):
        initializer = tf.random_normal_initializer(mean=0., stddev=1.,)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)
    
    def _bias_variable(self, shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)

In [22]:
if __name__ == '__main__':
    model = LSTMRNN(TIME_STEPS, INPUT_SIZE, OUTPUT_SIZE, CELL_SIZE, BATCH_SIZE)
    sess = tf.Session()
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs", sess.graph)
    init = tf.global_variables_initializer()
    sess.run(init)
    
    plt.ion()
    plt.show()
    for i in range(200):
        seq, res, xs = get_batch()
        if i == 0:
            feed_dict = {
                model.xs: seq,
                model.ys: res
                 # create initial state
            }
        else:
            feed_dict = {
                model.xs: seq,
                model.ys: res,
                model.cell_init_state: state
                # use last state as the initial state for this run
            }
        _, cost, state, pred = sess.run(
        [model.train_op, model.cost, model.cell_final_state, model.pred],
        feed_dict=feed_dict)
         # plotting
        plt.plot(xs[0, :], res[0].flatten(), 'r', xs[0,:], pred.flatten()[:TIME_STEPS])
        plt.ylim((-1.2, 1.2))
        plt.draw()
        plt.pause(0.3)
        
        if i % 20 == 0:
            print('cost: ', round(cost, 4))
            result = sess.run(merged, feed_dict)
            writer.add_summary(result, i)

ValueError: Variable in_hidden/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "d:\programdata\anaconda3\envs\tensor\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1756, in __init__
    self._traceback = tf_stack.extract_stack()
  File "d:\programdata\anaconda3\envs\tensor\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3322, in _create_op_internal
    op_def=op_def)
  File "d:\programdata\anaconda3\envs\tensor\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 742, in _apply_op_helper
    attrs=attr_protos, op_def=op_def)
  File "d:\programdata\anaconda3\envs\tensor\lib\site-packages\tensorflow_core\python\ops\gen_state_ops.py", line 1526, in variable_v2
    shared_name=shared_name, name=name)
  File "d:\programdata\anaconda3\envs\tensor\lib\site-packages\tensorflow_core\python\ops\state_ops.py", line 79, in variable_op_v2
    shared_name=sh